# Sudoku Solver with Linopy

The linopy package is good at tracking indices and writing optimization problems as vectorized functions (i.e., functions that operate on all rows of a column at once).  This notebook adapts the Medium article [Creating Sudoku Solver with Python and Pyomo in Easy Steps](https://medium.com/@dhanalakotamohan314/creating-sudoku-solver-with-python-and-pyomo-in-easy-steps-fe22ec916090) by Dhanalakota Mohan for linopy to demonstrate how indices (i.e., dimensions and coordinates) work.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import linopy

pd.options.display.float_format = "{:.0f}".format
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

# The Puzzle

Sudoku is a 9x9 grid where each of digits 1 through 9 appears once and only once in each row, columns, and 3x3 grid.  The puzzle is initiated with a values in some of the rows and columns, which I'm calling hints.  Here is an example Sudoku puzzle with hints as a Pandas dataframe.  

In [ ]:
# look at the hints for the puzzle
puzzle_hints = [
    (1, 7, 2),
    (2, 2, 8),
    (2, 6, 7),
    (2, 8, 9),
    (3, 1, 6),
    (3, 3, 2),
    (3, 7, 5),
    (4, 2, 7),
    (4, 5, 6),
    (5, 4, 9),
    (5, 6, 1),
    (6, 5, 2),
    (6, 8, 4),
    (7, 3, 5),
    (7, 7, 6),
    (7, 9, 3),
    (8, 2, 9),
    (8, 4, 4),
    (8, 8, 7),
    (9, 3, 6),
]
puzzle_hints = pd.DataFrame(puzzle_hints, columns=["row", "column", "digit"])
puzzle_hints_piv = puzzle_hints.pivot(
    index="row", columns="column", values="digit"
).replace(np.nan, "-")
puzzle_hints_piv

# Defining Dimensions and Coordinates in Xarray
In Pandas parlance, the `index` are the row labels and `columns` are the column labels.  When converting to Xarray, it is handy to name the index and columns with `df.index.name = "row_name"` and `df.columns.name = "column_name"`.  

In Xarray parlance, the Pandas `df.index.name` and `df.columns.name` are both called `dimensions`.  Then the array of values of the index and columns are called `coordinates`.  Think of latitude and longitude as `dimensions` with `coordinate` values.  Xarray will infer the dimensions and coordinates from a Pandas DataFrame, provided columns and the index have names.  The following cell shows how this looks.     

In [ ]:
xr.DataArray(puzzle_hints_piv)

In the Mohan article, the Sudoki puzzle is solved in a clever way.  Rather than select the digit for each (row, column) location in the grid, a third dimension is included which represents the digits 1, 2, 3, ... etc. that acts as an on-off switch for the value.  Here is what an empty (all zeros) Xarray DataArray will look like:    

In [ ]:
range_coord = range(1, 10)
xr.DataArray(
    np.zeros(shape=(9, 9, 9)),
    dims=["row", "column", "digit"],
    coords={"row": range_coord, "column": range_coord, "digit": range_coord},
)

And here is a graphical represenation:  

In [ ]:
data = np.random.rand(9, 9, 9)
array = xr.DataArray(data, dims=["column", "digit", "row"])


# Function to create a single cube at a specific location
def create_cube(x, y, z, size=1):
    # Define the vertices of a cube
    vertices = [
        [x, y, z],
        [x + size, y, z],
        [x + size, y + size, z],
        [x, y + size, z],
        [x, y, z + size],
        [x + size, y, z + size],
        [x + size, y + size, z + size],
        [x, y + size, z + size],
    ]
    # Define the 6 faces of the cube
    faces = [
        [vertices[j] for j in [0, 1, 2, 3]],
        [vertices[j] for j in [4, 5, 6, 7]],
        [vertices[j] for j in [0, 3, 7, 4]],
        [vertices[j] for j in [1, 2, 6, 5]],
        [vertices[j] for j in [0, 1, 5, 4]],
        [vertices[j] for j in [2, 3, 7, 6]],
    ]
    return faces


# Create a 3D plot
fig = plt.figure(figsize=(7, 4))
ax = fig.add_subplot(111, projection="3d")

# Colors for the slices
colors = plt.cm.viridis(np.linspace(0, 1, 9))

# Plot each cube
for i in range(9):
    for j in range(9):
        for k in range(9):
            faces = create_cube(i, j, k)
            poly3d = Poly3DCollection(
                faces, color=colors[j], edgecolor="k"
            )  # Color by digit dimension
            ax.add_collection3d(poly3d)

# Set custom labels
ax.set_xlabel("Column")
ax.set_ylabel("Digit")
ax.set_zlabel("")
ax.set_title("Sudoku Dimensions and Coordinates")

# Customize ticks and labels
ticks = np.arange(9)
labels = np.arange(1, 10)  # Labels from 1 to 9
ax.set_xticks(ticks)
ax.set_yticks(ticks)
ax.set_zticks([])  # Remove z-axis ticks
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

# Set the aspect ratio to be equal and adjust limits to fit the plot
ax.set_box_aspect([1, 1, 1])  # aspect ratio is 1:1:1
ax.set_xlim([0, 9])
ax.set_ylim([0, 9])
ax.set_zlim([0, 9])

# Adjust layout to ensure labels are visible
plt.subplots_adjust(left=0.5, right=0.8, top=0.9, bottom=0.0)

# Manually add z-axis labels
for z in range(9):
    ax.text(x=-1, y=-1, z=z + 0.5, s=9 - z)
ax.text(x=-4, y=-1, z=4.5, s="Row", ha="center")

plt.tight_layout()

## Indices for the 3x3 Squares

If you play Sudoku, you know that there are 3x3 squares that get each digit once and only once.  There we need to get indices for the 3x3 squares.  We will label each of the square indices within the three dimensions and make an xarray.  Later, within the optimization model, we will use the square indices to mask out portions of the puzzle.  This is how we are indexing the squares:   

In [ ]:
# Define the function to get the 3x3 square index
def get_square_index(row, col):
    return (row // 3) * 3 + (col // 3)


# Create the grid with row, column, and square indices
square_index = pd.DataFrame(
    {
        "row": np.repeat(range_coord, 9),
        "column": list(range_coord) * 9,
        "square": [
            get_square_index(row - 1, col - 1) + 1
            for row in range_coord
            for col in range_coord
        ],
    }
)
square_index = square_index.pivot(index="row", columns="column", values="square")
square_index

We put that in an xarray and then duplicate it across the third dimension "digit" for use in the model.  

In [ ]:
square_index = xr.DataArray(square_index)
square_index_3d = xr.concat([square_index] * len(range_coord), dim="digit")
square_index_3d = square_index_3d.assign_coords(digit=range_coord)
square_index_3d

# Model

Now we get to the model.  Initalize a Linopy model.  

In [ ]:
model = linopy.Model()

Make the binary `sudoku` variable for the model with the same dims and coords as above.

In [ ]:
sudoku = model.add_variables(
    name="sudoku", dims=square_index_3d.dims, coords=square_index_3d.coords, binary=True
)

It is handy to look at what Linopy is doing.  The code defined as single variable, but it is indexed on a 9x9x9 cube, so it is really 729 variables.  

In [ ]:
sudoku

Add the digit constraints to the model.  Each constraint sums across one of the dimensions, which holds the others constant for each summation.  By making the total == 1, the constrains ensure that the variable is turned on once and only once across that dimension.  In other words, we can't write a 1 and a 2 in the same square of the Sudoku puzzle.  The last constraint is displayed in Jupyter, which is helpful for seeing what is going on.  

In [ ]:
model.add_constraints(sudoku.sum(dim=["column"]) == 1, name="row_digit_constraint")
model.add_constraints(sudoku.sum(dim=["row"]) == 1, name="column_digit_constraint")
model.add_constraints(sudoku.sum(dim=["digit"]) == 1, name="row_column_constraint")

If you play Sudoku, you know there is also the 3x3 square constraint.  The general concept is the same as above, where we are setting the sum == 1 such that each digit occurs once and only once.  This is not quite as elegant as the code above because it has some loops and we use `square_index_3d` to select the right coordinates in the puzzle.  We do this with the `.where` function in Linopy.  

In [ ]:
for digit in range_coord:
    for square in range_coord:
        model.add_constraints(
            sudoku.where(square_index_3d == square).sel(digit=digit).sum() == 1,
            name=f"digit{digit}_square{square}_constraint",
        )

A Sudoku puzzle starts with some values already filled in.  These are also added to the model as constraints.  We add these by finding the right coordinates for each hint and making it == 1.  

In [ ]:
for _, datum in puzzle_hints.iterrows():
    model.add_constraints(
        sudoku.loc[dict(row=datum.row, column=datum.column, digit=datum.digit)] == 1,
        name=f"hint_{datum.row}_{datum.column}_{datum.digit}_constraint",
    )

I have been able to solve this without an objective, since the constraints lead to only one solution, but having an objective seems to help the solver.  

In [ ]:
model.add_objective(sudoku.sum(), sense="max")

# Solve

Solve the model with the default solver (HiGHs)

In [ ]:
model.solve()

The solution comes out as an xarray representing a cube of binary variables.  

In [ ]:
solution = model.solution.sudoku
solution

Squish the result down to two dimensions and put it in a DataFrame for display.  

In [ ]:
final_sudoku_grid = xr.zeros_like(solution.sel(digit=1), dtype=int)
for digit in solution.digit.values:
    final_sudoku_grid += digit * solution.sel(digit=digit).astype(int)

result = pd.DataFrame(
    data=final_sudoku_grid.values,
    columns=puzzle_hints_piv.columns,
    index=puzzle_hints_piv.index,
)
result

Looks like it was solved!  